# Laboratorium 6

Celem szóstego laboratorium jest zapoznanie się oraz zaimplementowanie algorytmu głębokiego uczenia aktywnego - REINFORCE. Zaimplementowany algorytm będzie testowany z wykorzystaniem środowiska z OpenAI - *CartPole*.


Dołączenie standardowych bibliotek

In [1]:
from collections import deque
import gym
import numpy as np
import random

Dołączenie bibliotek do obsługi sieci neuronowych

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.math import log
from tensorflow.math import reduce_sum
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

## Zadanie 1 - REINFORCE

<p style='text-align: justify;'>
Celem ćwiczenie jest zaimplementowanie algorytmu REINFORCE. Wagi sieci aktualizowane są zgodnie ze wzorem:
\begin{equation*}
    \theta \leftarrow \theta + \alpha G_t \nabla_\theta log \pi_{\theta}(a_t, s_t | \theta)
\end{equation*}.
</p>

In [3]:
def get_cumulative_rewards(rewards,  # rewards at each step
                           gamma=0.99  # discount for reward
                           ):
    """
    based on https://github.com/yandexdataschool/Practical_RL/blob/spring20/week06_policy_based/reinforce_tensorflow.ipynb
    take a list of immediate rewards r(s,a) for the whole session
    compute cumulative rewards R(s,a) (a.k.a. G(s,a) in Sutton '16)
    R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...

    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute R_t = r_t + gamma*R_{t+1} recurrently

    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    cumulative_rewards = []
    g = 0
    for r in reversed(rewards):
      g = r + gamma * g
      cumulative_rewards.append(g)
    cumulative_rewards.reverse()
    return cumulative_rewards

assert len(get_cumulative_rewards(range(100))) == 100
assert np.allclose(get_cumulative_rewards([0, 0, 1, 0, 0, 1, 0], gamma=0.9),
                   [1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards([0, 0, 1, -2, 3, -4, 0], gamma=0.5),
                   [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards([0, 0, 1, 2, 3, 4, 0], gamma=0), [0, 0, 1, 2, 3, 4, 0])

In [4]:
class REINFORCEAgent:
    def __init__(self, action_size, policy_model, value_model):
        self.G = 0
        self.action_size = action_size
        self.gamma = 0.99    # discount rate
        self.policy_model = policy_model
        self.value_model = value_model
        self.action_space = [i for i in range(action_size)]
        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []
        
    def remember(self, state, action, reward):
        #Function adds information to the memory about last action and its results
        self.state_memory.append(state)
        self.action_memory.append(action)
        self.reward_memory.append(reward)

    def get_action(self, state):
        """
        Compute the action to take in the current state, basing on policy returned by the network.

        Note: To pick action according to the probability generated by the network
        """

        #
        # INSERT CODE HERE to get action in a given state
        #        
        state = state[np.newaxis, :]
        chosen_action = np.random.choice(self.action_space, p=self.policy_model.predict(state)[0])

        return chosen_action

    def learn(self):
        """
        Function learn network using data stored in state, action and reward memory. 
        First calculates G_t for each state and train network
        """
        #
        # INSERT CODE HERE to train network
        #
        state_memory = np.array(self.state_memory)
        action_memory = np.array(self.action_memory)
        reward_memory = np.array(self.reward_memory)

        actions = np.zeros([len(action_memory), self.action_size])
        actions[np.arange(len(action_memory)), action_memory] = 1

        self.G = np.array(get_cumulative_rewards(reward_memory, self.gamma))
        self.value_model.train_on_batch([state_memory, self.G], actions)

        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []

Czas przygotować model sieci, która będzie się uczyła działania w środowisku [*CartPool*](https://gym.openai.com/envs/CartPole-v0/):

In [5]:
env = gym.make("CartPole-v0").env
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
learning_rate = 0.0005
   
#
# INSERT CODE HERE to build network, in the last layer use softmax activation function
#

input = Input(shape=(state_size,))
G = Input(shape=[1])
x1 = Dense(64, activation='relu')(input)
x2 = Dense(64, activation='relu')(x1)
probs = Dense(action_size, activation='softmax')(x2)

def custom_loss(y_true, y_pred):
    log_lik = y_true * log(y_pred)
    return reduce_sum(-log_lik * G)

value_model = Model(inputs=[input, G], outputs=[probs])
value_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=custom_loss)

policy_model = Model(inputs=[input], outputs=[probs])

Przygotuj funkcję obliczającą wartość nagrody skumulowanej:

Czas nauczyć agenta gry w środowisku *CartPool*:

In [6]:
agent = REINFORCEAgent(action_size, policy_model, value_model)


def generate_session(t_max=1000):
    """play env with REINFORCE agent and train at the session end"""

    reward = 0

    s = env.reset()

    for t in range(t_max):

        # chose action
        a = agent.get_action(s)

        new_s, r, done, info = env.step(a)

        # record session history to train later
        agent.remember(s, a, r)

        reward += r

        s = new_s
        if done: break

    agent.learn()

    return reward


for i in range(100):

    rewards = [generate_session() for _ in range(100)]  # generate new sessions

    print("mean reward:%.3f" % (np.mean(rewards)))

    if np.mean(rewards) > 300:
        print("You Win!")
        break

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


mean reward:32.310
mean reward:63.640
mean reward:118.650
mean reward:197.390
mean reward:331.510
You Win!
